In [12]:
import csv
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
import mpld3
import matplotlib.pyplot as plt
import matplotlib as mpl
import requests
import time
import datetime
import warnings
warnings.simplefilter('ignore')
import pymystem3
mystem = pymystem3.Mystem()
from __future__ import unicode_literals
from nltk.util import ngrams
from collections import Counter
#nltk.download('stopwords')
#nltk.download('punkt')
import nltk
from nltk.corpus import brown

In [13]:
################################################################################################################################
#table1 = pd.read_csv("./data_1.csv", encoding = 'utf-8',  sep=',') 
#table2 = pd.read_csv("./data2.csv", encoding = 'utf-8',  sep=',') 
#print(len(table1), len(table2))
#table = pd.concat([table1, table2]) 
table = pd.read_csv("./test.csv", encoding = 'utf-8',  sep=',') 
print(len(table))
table.drop_duplicates(keep = "first", inplace = True)
print(len(table))
#table.reset_index(inplace = True, drop = True)
#table = table.fillna('')

3039
3039


In [17]:
table = table.fillna('')
table.head(15)

,vk_id,vk_interests,vk_all_group_names,vk_all_group_descr,vk_fav_group_posts_list,vk_own_posts
0,907,,МАТЧ ТВ ФК «Зенит» Первый канал Летний лагерь ...,Официальное сообщество российского общедоступн...,#южныйгород @Самарский зоопарк ...,
1,976,,«Бумага» Лентач ПОСТ Высшая школа журналистики...,"Каждый день мы рассказываем о том, почему Пете...",Пятигорск Черногория Черногория ...,
2,1076,,,,,
3,2609,"Мода, музыка, искусство (живопись, фотография,...",Мозгократия. Давайте думать вместе Старинные и...,Дорогие друзья! Приглашаем Вас присоединиться ...,Новый уровень!Поздравь меня! Мои животные и ра...,
4,3693,,Настоящий Лентач KudaGo: Санкт-Петербург Атеис...,Проходите по нашим хештегам и легко находите ...,💥💥💥Поздравляем наших счастливчиков.\n1. @nadi_...,3 года без [id2797|Ани Тишинской]… 2 октября е...
5,4232,,Настоящий Лентач Химик - Психопат Я люблю русс...,"Химик-Психопат - мем, посвященный химикам и х...",Угадай счет — выиграй приз! Угадай счет — в...,как это было это я хорошечно в командировку пр...
6,5304,,Подслушано у учителей 5 лучших фильмов Типичны...,"Присылайте свои секретики о работе, рассуждени...",Кому не безразлична тайна фамилии Иванова http...,
7,5846,,Новости RT на русском Insane Studio SoSlack / ...,RT — это круглосуточный информационный телекан...,Офигеть танец..!!!я ...,Проектная работа для наставников.\n\nВ рамках...
8,6199,,Форум iPLACE: 30 октября 2018 SMMщики Питерски...,Все о менеджменте и управлении ресурсами на iP...,Хорошо разбираешься в Фильмах?! Давай провери...,Москва. Пальмира бизнес клуб \n\nTravel Winte...
9,6477,,IT Подслушано в МГУ - возможности для прогеров...,"Страница для разработчиков из МГУ.\nНе важно, ...","БЕСПЛАТНО! Курс "" Пробуждение женственности ""...",


In [18]:
sel_col = table[['vk_interests','vk_all_group_names','vk_own_posts','vk_fav_group_posts_list', 'vk_all_group_descr']].copy()
stop_words=[]
stop_words_ru = nltk.corpus.stopwords.words('russian')
stop_words_en = nltk.corpus.stopwords.words('english')
f1 = codecs.open('ru-stop.txt', 'r','utf-8')
stop_words = f1.read()
stop_words = stop_words.replace('\r', ' ').replace('\n', '')
stop_words = stop_words.split()
stop_words.extend(stop_words_ru)
stop_words.extend(stop_words_en)
stop_words= sorted(stop_words)

In [19]:
sel_col.head(10)

,vk_interests,vk_all_group_names,vk_own_posts,vk_fav_group_posts_list,vk_all_group_descr
0,,МАТЧ ТВ ФК «Зенит» Первый канал Летний лагерь ...,,#южныйгород @Самарский зоопарк ...,Официальное сообщество российского общедоступн...
1,,«Бумага» Лентач ПОСТ Высшая школа журналистики...,,Пятигорск Черногория Черногория ...,"Каждый день мы рассказываем о том, почему Пете..."
2,,,,,
3,"Мода, музыка, искусство (живопись, фотография,...",Мозгократия. Давайте думать вместе Старинные и...,,Новый уровень!Поздравь меня! Мои животные и ра...,Дорогие друзья! Приглашаем Вас присоединиться ...
4,,Настоящий Лентач KudaGo: Санкт-Петербург Атеис...,3 года без [id2797|Ани Тишинской]… 2 октября е...,💥💥💥Поздравляем наших счастливчиков.\n1. @nadi_...,Проходите по нашим хештегам и легко находите ...
5,,Настоящий Лентач Химик - Психопат Я люблю русс...,как это было это я хорошечно в командировку пр...,Угадай счет — выиграй приз! Угадай счет — в...,"Химик-Психопат - мем, посвященный химикам и х..."
6,,Подслушано у учителей 5 лучших фильмов Типичны...,,Кому не безразлична тайна фамилии Иванова http...,"Присылайте свои секретики о работе, рассуждени..."
7,,Новости RT на русском Insane Studio SoSlack / ...,Проектная работа для наставников.\n\nВ рамках...,Офигеть танец..!!!я ...,RT — это круглосуточный информационный телекан...
8,,Форум iPLACE: 30 октября 2018 SMMщики Питерски...,Москва. Пальмира бизнес клуб \n\nTravel Winte...,Хорошо разбираешься в Фильмах?! Давай провери...,Все о менеджменте и управлении ресурсами на iP...
9,,IT Подслушано в МГУ - возможности для прогеров...,,"БЕСПЛАТНО! Курс "" Пробуждение женственности ""...","Страница для разработчиков из МГУ.\nНе важно, ..."


In [21]:
%%time
p = codecs.open('dict_gensim_all_2.txt','w', 'utf-8')
x=[]
y=[]
s=[]
a=[]

tt = ''
unit_col = sel_col
I = len(sel_col)
J = len (sel_col.columns)
for i in range (737, 2000):
    tt = ''
    for j in range (J):
        tt = tt + (unit_col.iloc[i, j])
    word = nltk.word_tokenize(tt) # токинезация текстов i-го пользователя 

    word_ws = [w.lower()  for w in word if w.isalpha() ] # исключение слов и символов      

    word_w2 = [w for w in word_ws if w not in (stop_words)  ] #нижний регистр 
    lem = mystem.lemmatize((" ").join(word_w2)) # лемматизация i-го документа
    lema = [w for w in lem if w.isalpha() and len(w)>1]
    lema = [w for w in lema if w not in (stop_words)  ] #нижний регистр 

    #freq = nltk.FreqDist(lema)
    #z = [] # обновление списка для нового документа
    #z = [(key + ":" ) for key,val in freq.items() if val>1] # частота упоминания через : от слова    
    #z1 = [(key) for key,val in freq.items() if val>1]
    
    #для bigartm
    #p.write("|text" +" "+(" ").join(z) + "\r\n") # запись в мешок слов с меткой |text   
    
    #для gensim
    p.write((" ").join(lema) + "\r\n") # запись в мешок слов
#     list1 = freq.elements()
#     counts = dict(Counter(list1).most_common())
#     terms = list(counts.keys())
#     freqs = list(counts.values())
    
#     a = freqs[0:15]
#     b = terms[0:15]# TOP-10 для частот a  и слов b в i -м документе 
    
#     x.append(i)#список номеров документов
#     y.append(len(freq))#список количества слов в документах  
    
#     y_pos = np.arange(1,len(a)+1, 1)#построение TOP-10 диаграмм      
#     performance = a
#     plt.barh(y_pos, a)
#     plt.yticks(y_pos, b)
#     plt.xlabel(u'Количество слов')
#     plt.title(u'Частоты слов у юзверя № %i'%i, size = 10)
#     plt.grid(True)
#     plt.show()  
#     a[:] = []
#     b[:] = [] 
# plt.title(u'Количество слов в документах', size = 10)
# plt.xlabel(u'Номера документов', size = 10)
# plt.ylabel(u'Количество слов', size = 10)
# plt.bar(x,y, 1)
# plt.grid(True)
# plt.show()
p.close()  

Wall time: 23min 6s
